In [1]:
import igl
import math
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import os
root_folder = os.getcwd()

In [2]:
v_init, f = igl.read_triangle_mesh(os.path.join(root_folder, "oblate.off"))
global Kb
global Kv
global Ka
global gamma
gamma=10
Ka=2
Kv=1
Kb=0.01 
Volume_t= 0.7 * 3.14 * 4 / 3
Area_t=4*3.14

In [3]:
def Energy_area(v,f):
    Area_new=cal_Area(v,f)
    Area_t=4*3.14
    Eenergy_Area=Ka*((Area_new-Area_t)**2)/Area_t
    return Eenergy_Area
def Energy_volume(v,f):
    volume_old=cal_volume(v,f)
    volume_t=0.7*(4/3)*3.14
    Eenergy_volume=Kv*((volume_t-volume_old)**2)/volume_t
    return Eenergy_volume
def Energy_bending(v,f):
    k = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    kn = minv.dot(k) 
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

    minv = sp.sparse.diags(1 / m.diagonal())

    hn = -minv.dot(l.dot(v))
    h_mean = np.linalg.norm(hn, axis=1)
    #print(max(h_mean))
    #print(min(h_mean))
    c=len(v)
    #bending_modulus
    Eb=[] # bending_energy
    for i in range (c):
        Eb.append((Kb/2)*((h_mean[i])**2)*(area_voronoi[i]))
        Eb_array = np.array(Eb)
        total_EB=np.sum(Eb_array)
    return total_EB

In [4]:
def adjacent_face(v,f):
    df=pd.DataFrame(f,columns=list('ABC'))
    row_numbers=[]
    for i in range(len(v)):
        row_numbers.append((df.index[(df['A'] == i)|(df['B'] == i) | (df['C'] == i)].tolist()))
    return row_numbers

def cal_Area(v,f):
    dbl_area = igl.doublearea(v, f)    
    Area=np.sum(dbl_area)/2
    return Area
def volGrad(v,f):
    n = igl.per_vertex_normals(v, f)
    face_normal=igl.per_face_normals(v,f,n)
    dbl_area = igl.doublearea(v, f)/2
    adjacent_vertices=igl.adjacency_list(f)
    adjacent_faces=adjacent_face(v,f)
    volumegrad=[]
    for i in range(len(v)):
        vol_ij=0
        for j in range(len(adjacent_faces[i])):
            k=adjacent_faces[i][j]
            Area=dbl_area[k]
            FaceNorm=face_normal[k]
            vol_ij=vol_ij+((1/3)*Area*FaceNorm)
        volumegrad.append(vol_ij)
    volumegrad=np.array(volumegrad)
    return volumegrad

def areaGrad(v,f):
    n=igl.per_vertex_normals(v,f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    l = igl.cotmatrix(v, f) ###laplacian-operator
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    hn = -minv.dot(l.dot(v))
    h_mean = np.linalg.norm(hn, axis=1)
    areagrad=2*n*h_mean[:,None]
    return areagrad
def cal_volume(v,f):
    Volume=0
    for i in range(len(f)):
        sum=0
        p0x=v[f[i][0]][0]
        p0y=v[f[i][0]][1]
        p0z=v[f[i][0]][2]
        p1x=v[f[i][1]][0]
        p1y=v[f[i][1]][1]
        p1z=v[f[i][1]][2]
        p2x=v[f[i][2]][0]
        p2y=v[f[i][2]][1]
        p2z=v[f[i][2]][2]
        v321= p2x*p1y*p0z
        v231= p1x*p2y*p0z
        v312= p2x*p0y*p1z
        v132= p0x*p2y*p1z
        v213= p1x*p0y*p2z
        v123= p0x*p1y*p2z
        sum=(-v321+ v231+ v312-v132-v213+ v123) / 6.0
        #print(sum)
        Volume=Volume+sum
    return Volume

adjacent_faces=adjacent_face(v,f)
k=adjacent_faces[0][2]
n = igl.per_vertex_normals(v, f)
face_normal=igl.per_face_normals(v,f,n)
face_normal[k]

def fun_volgrad(v,f):
    n = igl.per_vertex_normals(v, f)
    #vector=np.array([1,1,1])
    #norm=vector/np.linalg.norm(vector)

    face_normal=igl.per_face_normals(v,f,n)
    dbl_area = igl.doublearea(v, f)/2
    volume_grad=face_normal*dbl_area[:,None]
    
    return volume_grad/3

In [5]:
def fun_ForceDensity(v,f):
    k = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    kn = minv.dot(k) 
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    hn = -minv.dot(l.dot(v))
    h_mean = np.linalg.norm(hn, axis=1)
    Lap_H=minv.dot(l.dot(h_mean/2))
    first_product=h_mean*(((h_mean/2)**2)-k)
    second_product=np.add(Lap_H,first_product)
    n = igl.per_vertex_normals(v, f)
    #normal_v=n/np.linalg.norm(n)
    ### Force Density and Nodal Force
    #Force_density=[] # Force_Density
    #Force_Nodal=[]

    #for i in range (len(v)):
    Force_density=2*Kb*(n*second_product[:,None])
        #Force_Nodal.append(Force_density[i]*area_voronoi[i])
        #print(Force_density[i]) 
        #print(Force_Nodal[i])
        
    return Force_density

In [6]:
def Force_Area(Area_t,grad_Area,Area_new):
    #grad_Area=volgrad(v,f)
    #Area_old=cal_volume(v,f)
    #Area_new=cal_Area(pos_new,f)
    Force_Area=-2*(Ka)*((Area_new-Area_t)/Area_t)*(grad_Area)
    return Force_Area

In [7]:
def Force_Volume(Volume_t,grad_Volume,volume_new):
    #grad_Volume=volgrad(v,f)
    #Volume_old=cal_volume(v,f)
    #Volume_new=cal_volume(pos_new,f)

    Force_Volume=-2*(Kv)*((volume_new-Volume_t)/Volume_t)*grad_Volume
    return Force_Volume

In [8]:
#Volume_old=cal_volume(v,f)
#Area_t=cal_Area(v,f)
grad_Area=areaGrad(v_init,f)
grad_Volume=volGrad(v_init,f)
Area_new=cal_Area(v_init,f)
volume_new=cal_volume(v_init,f)

In [9]:
###Dissipative Force
def fun_FD(v,f,velocity):
    adjacent_vertices=igl.adjacency_list(f)
    FDij_nodes=[]
    for i in range(len(v)):
        FD_ij=0
        for j in range(len(adjacent_vertices[i])):
            k=adjacent_vertices[i][j]
            vij=velocity[i]-velocity[k]
            rij=v[i]-v[k]
            #rij_distance=math.sqrt((rij[0][0]**2)+(rij[0][1]**2)+(rij[0][2]**2))
            rij_norm = rij/np.linalg.norm(rij)
            FD_ij=FD_ij+(-((gamma*np.dot(vij, rij)*rij_norm)))
        FDij_nodes.append(FD_ij) 
    return FDij_nodes

In [10]:
def Total_Force(FB,FA,FV,FD):
    Total_force=(FB+FA+FV+FB)
    return Total_force

grad_A=areaGrad(v,f)

grad_Volume=volgrad(v,f)
grad_Volume=np.array(grad_Volume)
grad_Volume.shape

In [ ]:
###Velocity Verlet Main 
dt=0.01
hdt=0.5*dt
iterations =2000
#Setup
v=v_init
vel=np.zeros((len(v_init),3))
FD=np.array(fun_FD(v_init,f,vel))
FB=np.array(fun_ForceDensity(v_init,f))
FA=Force_Area(Area_t,grad_Area,Area_new)
FV=Force_Volume(Volume_t,grad_Volume,volume_new)
TF=Total_Force(FB,FA,FV,FD)

df=open('iter=0.01_2000,gamma=10','w')
df.write('Total Energy of the System, dt=0.01\n')
for i in range(iterations):
    #Initial integration
    for j in range(len(v)):
        vel[j]+=0.5*TF[j]*dt
        v[j]+=vel[j]*dt
        #print(pos_new)
    if i%10==0:
        totalEnergy=Energy_area(v,f)+Energy_bending(v,f)+Energy_volume(v,f)
        df.write(str(totalEnergy))
        df.write('\n')
    
    #Force calculation
    Area_current=cal_Area(v,f)
    grad_Area_new=areaGrad(v,f)
    volume_current=cal_volume(v,f)
    grad_Volume_new=volGrad(v,f)
    
    ###Add forces_new here
    
    FD=np.array(fun_FD(v,f,vel))
    FB=np.array(fun_ForceDensity(v,f)) ##bending_force
    FA=Force_Area(Area_t,grad_Area_new,Area_current)
    
    FV=Force_Volume(Volume_t,grad_Volume_new,volume_current)
    TF=Total_Force(FB,FA,FV,FD)    
    
    #print(totalpressure_new)
    for j in range(len(v)):
        vel[j]+=0.5*TF[j]*dt
        #print(vel_new)
df.close()

In [ ]:
plot(v,f, shading={"wireframe": True}, return_plot=True)
v_init, f = igl.read_triangle_mesh(os.path.join(root_folder, "oblate.off"))
plot(v_init,f, shading={"wireframe": True}, return_plot=True)